<a href="https://colab.research.google.com/github/deepset-ai/haystack-experimental/blob/main/examples/async_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Haystack Pipelines in Asynchronous Environments

In this notebook, you'll learn how to use the `AsyncPipeline` and async-enabled components from the [haystack-experimental](https://github.com/deepset-ai/haystack-experimental) repository to build and execute a Haystack pipeline in an asynchronous environment. It's based on [this short Haystack tutorial](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline), so it would be a good idea to familiarize yourself with it before we begin. A further prerequisite is working knowledge of cooperative scheduling and [async programming in Python](https://docs.python.org/3/library/asyncio.html).

## Motivation

By default, the `Pipeline` class in `haystack` is regularly Python object class that exposes non-`async` methods to add/connect components and execute the pipeline logic. Currently, it *can* be used in async environments, but it's not optimal to do so since it executes it logic in a '[blocking](https://en.wikipedia.org/wiki/Blocking_(computing))' fashion, i.e., once the `Pipeline.run` method is invoked, it must run to completion and return the outputs before the next statement of code can be execute<sup>1</sup>. In a typical async environment, this prevents active async event loop from scheduling other `async` coroutines, thereby reducing throughput. Similarly, Haystack components currently only provide a non-`async` `run` method for their execution. To mitigate this bottleneck, we introduce the concept of async-enabled Haystack components and an `AsyncPipeline` class that cooperatively schedules the execution of both async and non-async components.

### Goals
- Allow individual components to opt into `async` support.
    - Not all components benefit from being async-enabled - I/O-bound components are the most suitable candidates.
- Provide a backward-compatible way to execute Haystack pipelines containing both async and non-async components.

### Non-goals
- Add async support to all existing components.
- Execute components concurrently.
    - While async support opens the door for concurrent execution, we're currently only focusing on providing basic `async` utility.

<sup>1</sup> - This is a simplification as the Python runtime can potentially schedule another thread, but it's a detail that we can ignore in this case.

Let's now go ahead and see what it takes to add async support to the original tutorial, starting with installing Haystack, the experimental package and the requisite dependencies.


In [ ]:
%%bash

pip install -U haystack-ai
pip install -U haystack-experimental
pip install datasets
pip install sentence-transformers

Provide an [OpenAI API key](https://platform.openai.com/api-keys) to ensure that LLM-based evaluators can query the OpenAI API.

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

# If you're running this notebook on Google Colab, you might need to the following instead:
#
# from google.colab import userdata
# if "OPENAI_API_KEY" not in os.environ:
#  os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Initialize a `DocumentStore` to index your documents. We use the `InMemoryDocumentStore` from the `haystack-experimental` package since it has support for `async`.

In [ ]:
from haystack_experimental.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

Fetch the data and convert it into Haystack `Document`s.

In [ ]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

To store your data in the `DocumentStore` with embeddings, initialize a `SentenceTransformersDocumentEmbedder` with the model name and call `warm_up()` to download the embedding model.

Then, we calculate the embeddings of the docs with the newly warmed-up embedder and write the documents to the document store. Notice that we call the `write_documents_async` method and use the `await` keyword with it. The `DocumentStore` protocol in `haystack-experimental` exposes `async` variants of common methods such as `count_documents`, `write_documents`, etc. These [coroutines](https://docs.python.org/3/library/asyncio-task.html#coroutines) are awaitable when invoked inside an async event loop (the notebook/Google Colab kernel automatically starts an event loop).

In [ ]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/all-MiniLM-L6-v2"
)
doc_embedder.warm_up()

docs_with_embeddings = doc_embedder.run(docs)
n_docs_written = await document_store.write_documents_async(docs_with_embeddings["documents"])
print(f"Indexed {n_docs_written} documents")

The next step is to build the RAG pipeline to generate answers for a user query.

Initialize a text embedder to create an embedding for the user query and an `InMemoryEmbeddingRetriever` to use with the `InMemoryDocumentStore` you initialized earlier. As with the latter, the async-enabled embedding retriever class stems from the `haystack-experimental` package.

In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_experimental.components.retrievers.in_memory import InMemoryEmbeddingRetriever

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = InMemoryEmbeddingRetriever(document_store)


Create a custom prompt to use with the `ChatPromptBuilder` and initialize a `OpenAIChatGenerator` to consume the output of the former.

In [ ]:
from haystack_experimental.components.builders import ChatPromptBuilder
from haystack_experimental.components.generators.chat import OpenAIChatGenerator
from haystack_experimental.dataclasses import ChatMessage

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = ChatPromptBuilder(template=[ChatMessage.from_user(template)])
generator = OpenAIChatGenerator(model="gpt-4o-mini")

We finally get to the creation of the pipeline instance. Instead of using the `Pipeline` class, we use the `AsyncPipeline` class from the `haystack-experimental` package. 

The rest of the process, i.e., adding components and connecting them with each other remains the same as with the original `Pipeline` class.

In [ ]:
from haystack_experimental.core import AsyncPipeline

async_rag_pipeline = AsyncPipeline()
# Add components to your pipeline
async_rag_pipeline.add_component("text_embedder", text_embedder)
async_rag_pipeline.add_component("retriever", retriever)
async_rag_pipeline.add_component("prompt_builder", prompt_builder)
async_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
async_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
async_rag_pipeline.connect("retriever", "prompt_builder.documents")
async_rag_pipeline.connect("prompt_builder.prompt", "llm.messages")

Now, we create a coroutine that queries the pipeline with a question.

The key differences between the `AsyncPipeline.run` and `Pipeline.run` methods have to do with their parameters and return values.

Both `Pipeline.run` and `AsyncPipeline.run` share the `data` parameter that encapsulates the initial inputs for the pipeline's components.

While `Pipeline.run` accepts an additional `include_outputs_from` parameter to return the outputs of intermediate, non-leaf components in the pipeline graph, `AsyncPipeline.run` does not. This is because the latter is implemented as an `async` generator that yields the output of **each component** as soon as it executes successfully. This has the following implications:

- The output of `AsyncPipeline.run` must be consumed in an `async for` loop for the pipeline execution to make progress.
- By providing the intermediate results as they are computed, it allows for a tighter feedback loop between the backend and the user. For example, the results of the retriever can be displayed to the user before the LLM's response is generated.

Whenever a component needs to be executed, the logic of `AsyncPipeline.run` will determine if it supports async execution. 
- If the component has opted into async support, the pipeline will schedule its execution as a coroutine on the event loop and yield control back to the async scheduler until the component's outputs are returned. 
- If the component has not opted into async support, the pipeline will launch its execution in a separate thread and schedule it on the event loop.

In both cases, given an `AsyncPipeline` only one of its components can be executing at any given time. However, this does not prevent multiple, different `AsyncPipeline` instances from executing concurrently.

The execution of an `AsyncPipeline` is deemed to be complete once program flow exits the `async for` loop. At this point, the final results of the pipeline (the outputs of the leaf nodes in the pipeline graph) can be accessed with the loop variable.

In [ ]:
from typing import Dict, Any


async def query_pipeline(question: str) -> Dict[str, Dict[str, Any]]:
    input = {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
    }

    result_idx = 0
    # The AsyncPipeline.run() method is an async generator that yields the output of each component.
    async for pipeline_output in async_rag_pipeline.run(input):
        print(f"Pipeline result '{result_idx}' = {pipeline_output}")
        result_idx += 1

    # The last output of the pipeline is the final pipeline output.
    return pipeline_output

We can now execute the pipeline with some examples.

In [ ]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "What does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
]

async def run_query_pipeline():
    global examples
    for question in examples:
        print(f"Querying pipeline with question: '{question}'")
        response = await query_pipeline(question)
        print(f'\tOutput: {response["llm"]["replies"][0]}\n')

    print("Done!")


await run_query_pipeline()

You can alternatively use the `run_async_pipeline` helper function to execute an `AsyncPipeline` in the same manner as a regular `Pipeline` while retaining the benefits of cooperative scheduling.

In [ ]:
from haystack_experimental.core import run_async_pipeline

question = examples[0]
outputs = await run_async_pipeline(
    async_rag_pipeline,
    {"text_embedder": {"text": question}, "prompt_builder": {"question": question}},
    include_outputs_from={"retriever"},
)

print(outputs)

## Custom Asynchronous Components

Individual components can opt into async by implementing a `run_async` coroutine that has the same signature, i.e., input parameters and outputs as the `run` method. This constraint is placed to ensure that pipeline connections the same irrespective of whether a component supports async execution, allowing for plug-n-play backward compatibility with existing pipelines.


In [ ]:
from typing import Dict, Any
from haystack import component

@component
class MyCustomComponent:
    def __init__(self, my_custom_param: str):
        self.my_custom_param = my_custom_param

    @component.output_types(original=str, concatenated=str)
    def run(self, input: str) -> Dict[str, Any]:
        return {
            "original": input,
            "concatenated": input + self.my_custom_param
        }

    async def do_io_bound_op(self, input: str) -> str:
        # Do some IO-bound operation here
        return input + self.my_custom_param

    @component.output_types(original=str, concatenated=str)
    async def run_async(self, input: str) -> Dict[str, Any]:
        return {
            "original": input,
            "concatenated": await self.do_io_bound_op(input)
        }